In [1]:
# https://towardsdatascience.com/hands-on-generative-adversarial-networks-gan-for-signal-processing-with-python-ff5b8d78bd28

In [2]:
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
from matplotlib import pyplot
import matplotlib.pyplot as plt

In [3]:
# https://stackoverflow.com/questions/68036975/valueerror-shape-must-be-at-least-rank-3-but-is-rank-2-for-node-biasadd
# config for rank error in lstm
tf.keras.backend.set_image_data_format("channels_last")

# https://stackoverflow.com/questions/58352326/running-the-tensorflow-2-0-code-gives-valueerror-tf-function-decorated-functio
# tf.config.run_functions_eagerly(True)

In [4]:
# %run ./read_file.ipynb

In [5]:
METRICS = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [6]:
batch_size = 32

In [7]:
input_len = 300

In [8]:
# define the standalone generator model
# https://www.tensorflow.org/tutorials/generative/dcgan#the_generator
def define_generator():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_len, 6)),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.Dense(6, activation='sigmoid'),
        tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1)),
    ])
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)
    return model

In [9]:
# define the standalone discriminator model
def define_discriminator():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(input_len, 6, 1)), # Flatten the input tensor
        tf.keras.layers.Embedding(100 + 1, 16),
        tf.keras.layers.Reshape((input_len, 6, 16)), # Reshape embedding output to match Dense output
        tf.keras.layers.Flatten(), # Flatten the output of the Embedding layer
        tf.keras.layers.Dense(input_len * 6, activation='tanh'), # Add an additional Dense layer to get desired output shape
        tf.keras.layers.Reshape((input_len, 6, 1)) # Reshape output to desired shape
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)
    return model

In [10]:
def gan_ex():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_len, 6, 1)),
        tf.keras.layers.Reshape((input_len, 6)), # Reshape input to 3D shape
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.Dense(6, activation='sigmoid'),
        tf.keras.layers.Reshape((input_len, 6, 1)), # Reshape output to 4D shape
        tf.keras.layers.Embedding(100 + 1, 16),
        tf.keras.layers.Reshape((input_len, 6, 16)), # Reshape embedding output to match Dense output
        tf.keras.layers.Flatten(), # Flatten the output of the Embedding layer
        tf.keras.layers.Dense(input_len * 6, activation='tanh'), # Add an additional Dense layer to get desired output shape
        tf.keras.layers.Reshape((input_len, 6, 1)) # Reshape output to desired shape
    ])
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)
    return model

In [11]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # connect them
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)
    return model

In [12]:
def generate_latent_space():
    n = tf.random.uniform(shape=[input_len, 6, 1], minval=1, maxval=100, dtype=tf.int32)
    n_ds = tf.reshape(n, (-1, input_len, 6, 1))
    return n_ds

In [13]:
def generate_fake_samples(generator, latent_dim=3, n=0):
    # generate points in latent space
    n_ds = generate_latent_space()
    # predict outputs
    print("size of input to generator")
    print(n_ds.shape)
    X = generator.predict(n_ds, verbose=0)
    # create class labels
    print("size of labels to discriminator")
    y = zeros((1, input_len, 6, 1, 1))
    print(y.shape)
    # add extra dimension to output tensor
    X = tf.expand_dims(X, axis=-1)
    return X, y

In [32]:
# use the generator to generate n fake examples, with class labels
def train(g_model, d_model, gan_model, latent_dim=3, n_epochs=5, n_batch=128, n_eval=200):
    # determine half the size of one batch, for updating the discriminator
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # prepare real samples
        x_real, y_real = generate_fake_samples(g_model, input_len)
        # prepare fake examples using the generator
        x_fake, y_fake = generate_fake_samples(g_model, input_len)
        # update discriminator
        d_model.train_on_batch(x_real, y_real)
        d_model.train_on_batch(x_fake, y_fake)
        # prepare points in latent space as input for the generator
        x_gan = generate_latent_space()
        # create inverted labels for the fake samples
        y_gan = ones((1, input_len, 6, 1))
        # create inverted labels for the fake samples
        # update the generator via the discriminator's error
        gan_model.train_on_batch(x_gan, y_gan)
        # evaluate the model every n_eval epochs
        if (i+1) % n_eval == 0:
            plt.title('Number of epochs = %i'%(i+1))
            pred_data = generate_fake_samples(generator, input_len)[0]
            real_data  = generate_fake_samples(generator, input_len)[0]
            plt.plot(pred_data[0],'.',label='Random Fake Sample',color='firebrick')
            plt.plot(real_data[0],'.',label = 'Random Real Sample',color='navy')
            plt.legend(fontsize=10)
            plt.show()

In [15]:
def train_single(g_model, d_model, gan_model, latent_dim=3, n_epochs=5, n_batch=128, n_eval=200):
    x_real, y_real = generate_fake_samples(generator, input_len)
    x_fake, y_fake = generate_fake_samples(generator, input_len)
    print("created from generator and trained in the discriminator")
    print("x_real" + str(x_real.shape))
    print("y_real" + str(y_real.shape))
    d_model.train_on_batch(x_real, y_real)
    d_model.train_on_batch(x_fake, y_fake)
    x_gan = generate_latent_space()
    # create inverted labels for the fake samples
    y_gan = ones((1, input_len, 6, 1))
    print("trained on the combination, input into the generator")
    print("x_gan " + str(x_gan.shape))
    print("y_gan " + str(y_gan.shape))
    gan_model.train_on_batch(x_gan, y_gan)

In [16]:
# Test combination of generator and discriminator
generator = define_generator()
discriminator = define_discriminator()
gan = define_gan(generator, discriminator)

In [17]:
# use generator

In [18]:
x_real, y_real = generate_fake_samples(generator, input_len)

size of input to generator
(1, 300, 6, 1)
size of labels to discriminator
(1, 300, 6, 1, 1)


In [19]:
x_real.shape

TensorShape([1, 300, 6, 1, 1])

In [20]:
y_real.shape

(1, 300, 6, 1, 1)

In [21]:
# use discriminator with generated data

In [22]:
discriminator.train_on_batch(x_real, y_real)

[0.01774211972951889, 0.0, 0.0, 1800.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]

In [23]:
# test gan created data and model

In [24]:
x_gan = generate_latent_space()
# x_gan = tf.reshape(x_gan, (-1, input_len, 6, 1, 1))
# create inverted labels for the fake samples
y_gan = ones(([1,input_len, 6, 1, 1]))

In [25]:
x_gan.shape

TensorShape([1, 300, 6, 1])

In [26]:
y_gan.shape

(1, 300, 6, 1, 1)

In [27]:
discriminator.train_on_batch(x_gan, y_gan)

[10.143481254577637, 0.0, 0.0, 0.0, 1800.0, 0.0, 0.0, 0.0, 0.0, 1.0]

In [28]:
gan.train_on_batch(x_gan, y_gan)

[15.124653816223145, 0.0, 0.0, 0.0, 3600.0, 0.0, 0.0, 0.0, 0.0, 1.0]

In [29]:
train_single(generator, discriminator, gan)

size of input to generator
(1, 300, 6, 1)
size of labels to discriminator
(1, 300, 6, 1, 1)
size of input to generator
(1, 300, 6, 1)
size of labels to discriminator
(1, 300, 6, 1, 1)
created from generator and trained in the discriminator
x_real(1, 300, 6, 1, 1)
y_real(1, 300, 6, 1, 1)
trained on the combination, input into the generator
x_gan (1, 300, 6, 1)
y_gan (1, 300, 6, 1)


In [33]:
train(generator, discriminator, gan)

size of input to generator
(1, 300, 6, 1)
size of labels to discriminator
(1, 300, 6, 1, 1)
size of input to generator
(1, 300, 6, 1)
size of labels to discriminator
(1, 300, 6, 1, 1)
size of input to generator
(1, 300, 6, 1)
size of labels to discriminator
(1, 300, 6, 1, 1)
size of input to generator
(1, 300, 6, 1)
size of labels to discriminator
(1, 300, 6, 1, 1)
size of input to generator
(1, 300, 6, 1)
size of labels to discriminator
(1, 300, 6, 1, 1)
size of input to generator
(1, 300, 6, 1)
size of labels to discriminator
(1, 300, 6, 1, 1)
size of input to generator
(1, 300, 6, 1)
size of labels to discriminator
(1, 300, 6, 1, 1)
size of input to generator
(1, 300, 6, 1)
size of labels to discriminator
(1, 300, 6, 1, 1)
size of input to generator
(1, 300, 6, 1)
size of labels to discriminator
(1, 300, 6, 1, 1)
size of input to generator
(1, 300, 6, 1)
size of labels to discriminator
(1, 300, 6, 1, 1)
